#### 100 med + voa sentences from 300 sentences

In [ ]:
import json
from collections import defaultdict

In [ ]:
with open('./data/MED.json') as f:
    med_data = json.loads(f.read())
    
with open('./data/voa_sentences.json') as f:
    voa_sentences = json.loads(f.read())

In [ ]:
sent2ans = {}
med_sents = {}
word_sense = defaultdict(list)
for word in voa_sentences:
    for sense in med_data[word]['noun']['SENSE']:
        definition = sense[1] + ' ' + sense[0]
        examples = sense[-1]
        word_sense[word].append(definition)
        for sent in examples:
            sent2ans[sent[0]] = definition 
        med_sents[definition] = sorted([sent[0] for sent in examples],
                                        key=lambda x: len(x.split()), 
                                        reverse=True)

In [ ]:
import math

In [ ]:
with open('./data/300_sentences.json') as f:
    evaluation_300 = json.loads(f.read())

In [ ]:
TARGET_NUM = 120
WORDS_LIST = list(voa_sentences.keys())

med_sentences = {}
evaluation_sents = defaultdict(list)
AVG_NUM = math.ceil(TARGET_NUM / len(WORDS_LIST))


for word, examples in evaluation_300.items():
    med_examples = [[sent, sent2ans[sent]] 
                   for sent in examples if sent in sent2ans]
    med_sentences[word] = med_examples
    
    # deal with words that do not have examples
    if not med_examples:
        # if all voa_sentences do not enough then we add all sentences
        if len(voa_sentences[word]) <= AVG_NUM:
            evaluation_sents[word] = [[sent.replace('\n', ''), '']
                                      for sent in voa_sentences[word]]
            TARGET_NUM -= len(evaluation_sents[word])
            WORDS_LIST.remove(word)
        else:
            voa_examples = [[sent.replace('\n', '' ), '']
                            for sent in voa_sentences[word]]
            evaluation_sents[word] = random.sample(voa_examples, 10)
            TARGET_NUM -= len(evaluation_sents[word])
            WORDS_LIST.remove(word)

for word in WORDS_LIST:
    examples = med_sentences[word]
    if len(examples) <= AVG_NUM:
        left = AVG_NUM - len(examples)
        evaluation_sents[word] = examples
        if left > 0:
            voa_examples = [[sent.replace('\n', ''), ''] 
                            for sent in voa_sentences[word]]
            if len(voa_examples) > left:
                voa_examples = random.sample(voa_examples, left)
            evaluation_sents[word].extend(voa_examples)
    else:
        temp = AVG_NUM
        while temp > 0:
            for sense in word_sense[word]:
                examples = med_sents[sense]
                if examples:
                    sent = examples[0]
                    ans = sent2ans[sent]
                    evaluation_sents[word].append([sent, ans])
                    med_sents[sense] = examples[1:]
                    temp -= 1

In [ ]:
"""
check total number of sentences
"""
def check_count(dictionary):
    count = 0
    for key, value in dictionary.items():
        print(key, len(value))
        count += len(value) 
    print(count)

In [ ]:
check_count(evaluation_sents)

In [ ]:
# with open('./data/100_sentences.json', 'w') as f:
#     f.write(json.dumps(evaluation_sents))

### Other 200 sentences

In [ ]:
with open('./data/100_sentences.json') as f:
    evaluation_sents = json.loads(f.read())

In [ ]:
other_200sents = defaultdict(list)

WORDS_LIST = list(voa_sentences.keys())
TARGET_NUM = 200
AVG_NUM = math.ceil(TARGET_NUM / len(WORDS_LIST))

exists_sents = {word: [sent[0] for sent in evaluation_sents[word]] 
                for word in evaluation_sents}
max_count = {}
for word in WORDS_LIST:
    voa_sent = [sent.replace('\n', '') for sent in voa_sentences[word]
                if sent.replace('\n', '') not in exists_sents[word]]
    max_count[word] = len(voa_sent)
    
for word in voa_sentences:
    if max_count[word] < AVG_NUM:
        voa_sent = [sent.replace('\n', '') for sent in voa_sentences[word]
                if sent.replace('\n', '') not in exists_sents[word]]
        WORDS_LIST.remove(word)
        TARGET_NUM -= max_count[word]
        other_200sents[word] = voa_sent
        

AVG_NUM = math.ceil(TARGET_NUM / len(WORDS_LIST))
for word in WORDS_LIST:
    voa_sent = [[sent.replace('\n', ''), ''] for sent in voa_sentences[word]
                if sent.replace('\n', '') not in exists_sents[word]]
    examples = random.sample(voa_sent, AVG_NUM)
    other_200sents[word].extend(examples)

In [ ]:
check_count(other_200sents)

In [ ]:
# with open('./data/200_sentences.json', 'w') as f:
#     f.write(json.dumps(other_200sents))